In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

ds = pd.read_csv("AR_all.csv")
ds=ds[0:11]
print(ds[[ 'name','id','sku']])

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df=1, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['name'])
print(tf.get_feature_names())
#print(tfidf_matrix)
#print(ds['name'][0:5])
tf1 = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df=1, stop_words='english')
tfidf_matrix1 = tf1.fit_transform(ds['sku'])
print(tf1.get_feature_names())
#print(tfidf_matrix1)

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_similarities1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)
#print(cosine_similarities)
results = {}
results1 = {}

for idx, row in ds.iterrows():
    #print('',)
    similar_indices = cosine_similarities[idx].argsort()[::-1]
   # print(similar_indices)
    similar_items = [(cosine_similarities[idx][i], ds['id'][i]) for i in similar_indices]
    similar_indices1 = cosine_similarities1[idx].argsort()[::-1]
    #print(similar_indices1)
    similar_items1 = [(cosine_similarities1[idx][i], ds['id'][i]) for i in similar_indices1]
    #print(similar_items1)
    # First item is the item itself, so remove it.
    # Each dictionary entry is like: [(1,2), (3,4)], with each tuple being (score, item_id)
    results[row['id']] = similar_items[1:]
    results1[row['id']] = similar_items1[1:]  
print('done!')

                                                name  id  \
0   Coffee coco butter ylang ylang soap 125gm (Copy)   4   
1        OW' Real Tulsi Green Tea + Saffron -25 bags   5   
2      Aum Fresh Sundarikalp Tea 50 gm (25 Tea Bags)  10   
3                  Baby natural laundry wash 1000 ml  11   
4            Kairali Amruthotharam Kashayam (200 ml)  12   
5         Aum Fresh Nidrakar Tea 50 gm (25 Tea Bags)  13   
6           Aum Fresh Memory Tea 50 gm (25 Tea Bags)  14   
7           Kairali Balaguluchyedi Kashayam (200 ml)  15   
8       Aum Fresh Gas Relief Tea 50 gm (25 Tea Bags)  16   
9         Aum Fresh Vajikaran Tea 50gm (25 Tea Bags)  17   
10         Kairali Cheriya Rasnadi Kashayam (200 ml)  18   

                                              sku  
0                                        SKU 9480  
1                                       SKU 11067  
2   Aum Fresh Sundarikalp Tea 50 gm (25 Tea Bags)  
3               Baby natural laundry wash 1000 ml  
4         Kairali A

In [4]:
def item(id):
    return ds.loc[ds['id'] == id]['name'].tolist()[0].split(' - ')[0]

# Just reads the results out of the dictionary. No real logic here.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    recs1 = results1[item_id][:num]
    print(recs1)
    l=[]
    l1=[]
    #Sprint(recs)
    for i in recs:
        l.append(i[1])
    print(l)     
    for rec in recs:
       if rec[0]>(0.50/2):
         print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")
    for i in recs1:
        l1.append(i[1])
    print(l1)     
    for rec in recs1:
       if rec[0]>0.50:
         print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")    

# Just plug in any item id here (1-500), and the number of recommendations you want (1-99)
# You can get a list of valid item IDs by evaluating the variable 'ds', or a few are listed below

recommend(item_id=10, num=10)

Recommending 10 products similar to Aum Fresh Sundarikalp Tea 50 gm (25 Tea Bags)...
-------
[(0.7936077671566899, 14), (0.7936077671566899, 13), (0.7225395819469875, 16), (0.601304817994677, 17), (0.0, 18), (0.0, 15), (0.0, 12), (0.0, 11), (0.0, 5), (0.0, 4)]
[14, 13, 16, 17, 5, 18, 15, 12, 11, 4]
Recommended: Aum Fresh Memory Tea 50 gm (25 Tea Bags) (score:0.775864421749223)
Recommended: Aum Fresh Nidrakar Tea 50 gm (25 Tea Bags) (score:0.775864421749223)
Recommended: Aum Fresh Gas Relief Tea 50 gm (25 Tea Bags) (score:0.7012470979466564)
Recommended: Aum Fresh Vajikaran Tea 50gm (25 Tea Bags) (score:0.5674002443191339)
[14, 13, 16, 17, 18, 15, 12, 11, 5, 4]
Recommended: Aum Fresh Memory Tea 50 gm (25 Tea Bags) (score:0.7936077671566899)
Recommended: Aum Fresh Nidrakar Tea 50 gm (25 Tea Bags) (score:0.7936077671566899)
Recommended: Aum Fresh Gas Relief Tea 50 gm (25 Tea Bags) (score:0.7225395819469875)
Recommended: Aum Fresh Vajikaran Tea 50gm (25 Tea Bags) (score:0.601304817994677)


In [35]:
import re
numbers = "(^a(?=\s)|one|two|three|four|five|six|seven|eight|nine|ten| \
          eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen| \
          eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty| \
          ninety|hundred|thousand|([0-9]*[.])?[0-9]+)"


for i in ds['name']:
    if re.search(r"tea",i, re.IGNORECASE):
       x = re.findall(numbers, i)
       print(i)
       print((x[0][0]))

OW' Real Tulsi Green Tea + Saffron -25 bags
25
Aum Fresh Sundarikalp Tea 50 gm (25 Tea Bags)
50
Aum Fresh Nidrakar Tea 50 gm (25 Tea Bags)
50
Aum Fresh Memory Tea 50 gm (25 Tea Bags)
50
Aum Fresh Gas Relief Tea 50 gm (25 Tea Bags)
50
Aum Fresh Vajikaran Tea 50gm (25 Tea Bags)
50


In [14]:
import docx2txt
import re
#my_text = docx2txt.process("profilesforproject/2.docx")
#print(type(my_text))
#print(re.split('.\n',my_text))
#my_text.replace('\t', '\n')
#print(my_text)
#print(my_text.split('.\n'))
#print(re.split('.\n|.\t',my_text))

In [147]:
import docx2txt
import re
import textract
my_text = textract.process("profilesforproject/5.doc")
my_text = text.decode("utf-8") 
#my_text = docx2txt.process("profilesforproject/bajeeDotdevops@outlookDotcom.docx")
#my_text=my_text.lower().split('.\n')
print(type(my_text))
#print(my_text)
for i in my_text:
    print('ffffffffffff',i)

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
example_sent = "This is a sample sentence, showing off the stop words filtration."
  
stop_words = set(stopwords.words('english')) 
  
word_tokens = word_tokenize(example_sent) 
  
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
  
#print(word_tokens) 
#print(filtered_sentence)
ll=' '.join(filtered_sentence)
#print(ll.lower().split("."))
#print(my_text.split(".\t\n"))

AttributeError: 'str' object has no attribute 'decode'

In [145]:
import textract
kk=['mvc','vb.net','c#.net','.net']
exp=['experience','knowledge','idea']
exp1=[]
text = textract.process("profilesforproject/5.doc")
text = text.decode("utf-8")
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()
word = "playing" 
print(lem.lemmatize(word))

#print(text)
for i in exp:
  exp1.append(lem.lemmatize(i))
print(exp1)    
text=text.replace(',', '')
#print(text[15])
my_text=text.lower().split('.\n')
#print(my_text[15])

ll=[]
for i in my_text:
 ll.append(i.split())
ll2=[]
for i in ll:
    j1=[]
    for j in i:
       l=lem.lemmatize(j) 
       j1.append(l)
    ll2.append(j1)    
 
for i in ll2:
    print(i)
    for j in i:
       for l in kk: 
        if j==l:
            for k in i:
                if k=='year':
                  print(j,i[i.index(k)-1],'yyyyy') 
                #if k=='experience':
                if k in exp1:
                    print(j,'knowledge')

playing
['experience', 'knowledge', 'idea']
['|buchha', 'rao.padidhala', '|', '|', '|ph:', '(m)', '+91-9493812725', '|', '|', '|e-mail:padithala@gmail.com', '|', '|', 'objective:', 'to', 'obtain', 'a', 'challenging', 'position', 'in', 'the', 'field', 'of', 'it', 'with', 'a', 'progressive', 'organization', 'that', 'will', 'fully', 'utilize', 'my', 'talent', 'and', 'technical', 'skill', 'and', 'help', 'me', 'reach', 'higher', 'echelon']
['synopsis:', 'meticulous', 'and', 'highly', 'accomplished', 'professional', 'with', 'around', '4+', 'year', 'of', 'experience', 'a', 'software', 'engineer.', 'extended', 'expertise', 'in', '.net', '3.5', '4.04.5', 'c#.net', 'vb.netasp.net', 'asp.net', 'mvcweb', 'service', 'javascript', 'jquery', 'sql', 'server', '20012linqentityframeowrk', 'ms-office']
.net 4+ yyyyy
.net knowledge
c#.net 4+ yyyyy
c#.net knowledge
['<', 'proven', 'ability', 'to', 'produce', 'prioritize', 'needs/risks', 'and', 'develop', 'solution', '<', 'through', 'understanding', 'of', '

In [18]:
import glob
import textract
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df=1, stop_words='english')
ds=[]
final={}
main={}
#print(tf.get_feature_names())
for i in glob.glob("/Users/pradeep/Documents/jupyter/profilesforproject/*"):
    text = textract.process(i)
    name=(i.split('/')[-1]).split('.')[0]
    final[name]=text
    text = text.decode("utf-8")
    #print(text)
    ds.append(text)
from pyresparser import ResumeParser
data = ResumeParser('/Users/pradeep/Documents/jupyter/profilesforproject/1.docx').get_extracted_data()    
#description='python 2.5 years of experience and java'
technologies=['java','python','.net','jquery','c#.net','mvc','jquery','javascript','python','flask','sql']
#description=' '.join(data['skills']

description=' '.join(technologies)
final['description']=description

ds.append(description)    
for i,v in enumerate(final.keys()):
    main[i]=v
tfidf_matrix = tf.fit_transform(final.values()) 
#print(tf.get_feature_names())
cosine_similarities = linear_kernel(tfidf_matrix[-1], tfidf_matrix)
#print(cosine_similarities[0][::-1])
for i,v in enumerate(cosine_similarities[0]):
    if v > 0:
      print(main[i],round(v*100,3),'%')
    

jnjhjhjhjhjh 0 0.22743282825637384
4 22.743 %
jnjhjhjhjhjh 1 0.20459489879580783
1 20.459 %
jnjhjhjhjhjh 2 0.12235553676090287
mani1 12.236 %
jnjhjhjhjhjh 3 0.29625463845809485
5 29.625 %
jnjhjhjhjhjh 4 0.12675809072453637
2 12.676 %
jnjhjhjhjhjh 5 0.14822704447281082
3 14.823 %
jnjhjhjhjhjh 6 0.014324219912915688
diff 1.432 %
jnjhjhjhjhjh 7 0.12239443648174252
mani2 12.239 %
jnjhjhjhjhjh 8 1.0


In [2]:
from pyresparser import ResumeParser
data = ResumeParser('/Users/pradeep/Documents/jupyter/profilesforproject/1.docx').get_extracted_data()
print(data['skills'])
technologies=['java','python','.net','jquery','c#.net','mvc','jquery','javascript','python','flask','sql']
ll=[]
for i in technologies:
    if i in data['skills']:
        ll.append(i)
print(ll)        

['Mysql', 'Asp', 'Technical skills', 'Website', 'Windows', 'Communication', 'Construction', 'Scripting', 'Html5', 'C', 'Js', 'Analytical', 'Salesforce', 'System', 'Architecture', 'Process', 'English', 'Reports', 'C#', 'Agile', 'Writing', 'Sql server', 'Tfs', 'Database', 'Sql', 'Queries', 'Api', 'Controls', 'Javascript', 'Technical', 'Operations', 'Test cases', 'Json', 'Engineering', 'Computer science', '.net', 'Testing', 'Analysis', 'Design', 'Visual']
['.net']


In [3]:
import glob
import textract
str1='i have 2 years off experience in python and 4 years of experience in java'
str2='i have 3 years of experience in .net and jquery'
str3='i have high knowledge in python and basic knowledge in java'
str4='i have basic knowledge in c#.net but no knowledge in mvc'
str5='i have total of 10 years of experience in c#.net of which 2 years .net1.0 and 8 years in .net2.0  '
technologies=['java','python','.net','jquery','c#.net','mvc','.net1.0','.net2.0']
exp=['experience','knowledge']
year=['year','month']
levels={'basic':2,'no':0,'high':8}
ll2=str4.split(' ')
result={}
ll=[]
for k,j in enumerate(ll2):
       #print(j)
       for l in technologies: 
        #print(k,l)    
        if j==l:
            flag1='TRUE'
            flag2='TRUE'
            front=ll2[:k+1]
            rare=ll2[k+1:]
            for k1,k in enumerate(front[::-1]):
                if k=='years' and flag1=='TRUE':  
                  #print(j,front[-k1-2],front[-k1-1]) 
                  ll.append((j,front[-k1-2]+' '+front[-k1-1]))  
                  flag1='FLASE'
                if k in exp and flag2=='TRUE':
                  #print(j,front[front.index(k)-1],front[front.index(k)]) 
                  ll.append((j,front[-k1-2]+' '+front[-k1-1]))
                  flag2='FLASE' 
                    
            for r,k in enumerate(rare):
                #print('k',k)
                if k=='years' and flag1=='TRUE':
                  #print(j,rare[r])
                  ll.append((j,rare[r-1]+' '+rare[r]))
                  flag1='FLASE'
                if k in exp and flag2=='TRUE':
                  #print(j,front[front.index(k)-1],front[front.index(k)]) 
                  ll.append((j,rare[r-1]+' '+rare[r]))
                  flag2='FLASE'                     
                                                     
print(ll) 


[('c#.net', 'basic knowledge'), ('mvc', 'no knowledge')]


In [32]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()
import glob
import textract
#print(porter.stem('javascript'))
str1='i have 2 years off experience in python and 4 years of experience in java'
str2='i have 3 years of experience in .net and jquery'
str3='i have high knowledge in python and basic knowledge in java'
str4='i have basic knowledge in c#.net but no knowledge in mvc'
technologies=['java','python','.net','jquery','c#.net','mvc','jquery','javascript','python','flask','sql']
exp=['experience','knowledge','idea','language','develped','involved']
year=['year','month','years','months']
exp1=[]
year1=[]
technologies1=[]
for k in exp:
    exp1.append(porter.stem(k))
for k in year:
    year1.append(porter.stem(k))
for k in technologies:
    technologies1.append(porter.stem(k))   
print('exp  :-',exp1,'\n','yeras :-',year1,'\n','technologies :-',technologies1,'\n\n\n')    
ll2=str4.split(' ')
result={}
ll=[]
text = textract.process("profilesforproject/5.doc")
text = text.decode("utf-8")
text=text.replace(',', ' ').replace('(',' ').replace(')',' ').replace(':',' ')
#print(text[15])
my_text=text.lower().split('.\n')
#print(my_text[15])

ll3=[]
for i in my_text:
 kk=i.split()
 kk1=[]
 for term in kk:
      kk1.append(porter.stem(term))  
 ll3.append(kk1)
        
for ll2 in ll3:
 #print('ssssss',ll2)   
 for k,j in enumerate(ll2):
       #print('ooooo',j)
       for l in technologies1: 
        #print('oooooooooo',k,j,l)    
        if j==l:
            flag1='TRUE'
            flag2='TRUE'
            front=ll2[:k+1]
            rare=ll2[k+1:]
            #print('kkkkkkkkkkk',front,rare)
            for k1,k2 in enumerate(front[::-1]):
                if k2 in year1 and flag1=='TRUE':  
                  #print(j,front[-k1-2],front[-k1-1]) 
                  ll.append((j,front[-k1-2]+' '+front[-k1-1]))  
                  flag1='FLASE'
                if k2 in exp1 and flag2=='TRUE':
                  #print(j,front[front.index(k)-1],front[front.index(k)]) 
                  #ll.append((j,'knowledge'))
                  ll.append((j,front[-k1-2]+' experience'))
                  flag2='FLASE' 
                    
            for r,k4 in enumerate(rare):
                #print('k',k)
                if k4 in year1 and flag1=='TRUE':
                  #print(j,rare[r])
                  ll.append((j,rare[r-1]+' '+rare[r]))
                  flag1='FLASE'
                if k4 in exp1 and flag2=='TRUE':
                  #print(j,front[front.index(k)-1],front[front.index(k)]) 
                  #ll.append((j,'knowledge'))
                  ll.append((j,rare[r-1]+' experience'))                  
                  flag2='FLASE'                     
                                                     
#print(ll) 
for key,value in ll:
    if key not in result.keys():
        result[key]=[value]
    else:
        result[key].append(value)
print(result)        

exp  :- ['experi', 'knowledg', 'idea', 'languag', 'develp', 'involv'] 
 yeras :- ['year', 'month', 'year', 'month'] 
 technologies :- ['java', 'python', '.net', 'jqueri', 'c#.net', 'mvc', 'jqueri', 'javascript', 'python', 'flask', 'sql'] 



{'.net': ['of experience', '4+ year', '2+ year', 'total experience'], 'c#.net': ['of experience', '4+ year', 'total experience', '2+ year', '1+ year', 'year experience'], 'mvc': ['of experience', '4+ year', '1 year', 'total experience'], 'javascript': ['of experience', '4+ year'], 'jqueri': ['of experience', '4+ year', 'of experience', '4+ year', '6 month', 'total experience', '6 month', 'total experience'], 'sql': ['of experience', '4+ year', 'of experience', '6+ month']}


In [ ]:
this is main for all 

In [78]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()
import glob
import textract
#print(porter.stem('javascript'))
str1='i have 2 years off experience in python and 4 years of experience in java'
str2='i have 3 years of experience in .net and jquery'
str3='i have high knowledge in python and basic knowledge in java'
str4='i have basic knowledge in c#.net but no knowledge in mvc'
technologies=['java','python','.net','jquery','c#.net','mvc','jquery','javascript','python','flask','sql','asp.net','c#']
exp=['experience1','knowledge','idea','language','develped','involved','machinelearning','artificialinteligence',]
year=['year','month','years','months']
words=['machine learning','artificial inteligence','java script','j query',]
shortcuts={'ai':'artificialinteligence','ml':'machinelearning','nlp':'naturallanguageprocessing'}
exp1=[]
year1=[]
technologies1=[]
for k in exp:
    exp1.append(porter.stem(k))
for k in year:
    year1.append(porter.stem(k))
for k in technologies:
    technologies1.append(porter.stem(k))   
print('exp  :-',exp1,'\n','yeras :-',year1,'\n','technologies :-',technologies1,'\n\n\n')    
ll2=str4.split(' ')
files=[]
final={}
job=textract.process("/Users/pradeep/Documents/jupyter/job_description.docx")
job = job.decode("utf-8")
job=job.replace(',', ' ').replace('(',' ').replace(')',' ').replace(':',' ').replace('+',' ')
job1=[]
job=job.lower().split(' ')
for i in job:
    if i in technologies:
      job1.append(porter.stem(i))
print(job1)    

for file in glob.glob("/Users/pradeep/Documents/jupyter/profilesforproject/*"):
    result={}
    ll=[]
    text = textract.process(file)
    name=(file.split('/')[-1]).split('.')[0]
    text = text.decode("utf-8")

    text=text.replace(',', ' ').replace('(',' ').replace(')',' ').replace(':',' ').replace('+',' ')
#print(text[15])
    my_text=text.lower().split('.\n')
#print(my_text[15])

    ll3=[]
    for i in my_text:
     kk=i.split()
     kk1=[]
     for term in kk:
          if term in shortcuts.keys():
           kk1.append(porter.stem(shortcuts[term]))
          else:
           kk1.append(porter.stem(term))

                
     ll3.append(kk1)
        
    for ll2 in ll3:
     #print('ssssss',ll2)   
     for k,j in enumerate(ll2):
           #print('ooooo',j)
           for l in job1: 
            #print('oooooooooo',k,j,l)    
            if j==l:
                flag1='TRUE'
                flag2='TRUE'
                front=ll2[:k+1]
                rare=ll2[k+1:]
                #print('kkkkkkkkkkk',front,rare)
                for k1,k2 in enumerate(front[::-1]):
                    if k2 in year1 and flag1=='TRUE':  
                      #print(j,front[-k1-2],front[-k1-1]) 
                      ll.append((j,front[-k1-2]+' '+front[-k1-1]))  
                      flag1='FLASE'
                    if k2 in exp1 and flag2=='TRUE':
                      #print(j,front[front.index(k)-1],front[front.index(k)]) 
                      #ll.append((j,'knowledge'))
                      #print(j,front[-k1-2])  
                      ll.append((j,'experience'))
                      flag2='FLASE' 

                for r,k4 in enumerate(rare):
                    #print('k',k)
                    if k4 in year1 and flag1=='TRUE':
                      #print(j,rare[r])
                      ll.append((j,rare[r-1]+' '+rare[r]))
                      flag1='FLASE'
                    if k4 in exp1 and flag2=='TRUE':
                      #print(j,front[front.index(k)-1],front[front.index(k)]) 
                      #print(j,rare[r-1])
                      #ll.append((j,'knowledge'))
                      ll.append((j,'experience'))                  
                      flag2='FLASE'                     
                                                     
    #print(ll) 
    for key,value in ll:
        if key not in result.keys():
            result[key]=[value]
        else:
            rr=result[key]
            rr.append(value)
            result[key]=list(set(rr))
    #print(result) 
    final[name]=result
print(final) 
import pandas as pd
#print(pd.DataFrame.from_dict(final, orient='index'))
data=pd.DataFrame.from_dict(final)
print(data,data.to_dict())

exp  :- ['experience1', 'knowledg', 'idea', 'languag', 'develp', 'involv', 'machinelearn', 'artificialintelig'] 
 yeras :- ['year', 'month', 'year', 'month'] 
 technologies :- ['java', 'python', '.net', 'jqueri', 'c#.net', 'mvc', 'jqueri', 'javascript', 'python', 'flask', 'sql', 'asp.net', 'c#'] 



['python', 'python', 'python', 'python', 'flask', 'javascript']
{'4': {'javascript': ['experience']}, '1': {'javascript': ['experience']}, 'mani1': {'python': ['experience'], 'flask': ['experience']}, '5': {'javascript': ['4 year']}, '2': {}, '3': {'javascript': ['experience', '4.7 year']}, 'diff': {}, 'mani2': {'python': ['experience'], 'flask': ['experience']}}
                       4             1         mani1         5   2  \
javascript  [experience]  [experience]           NaN  [4 year] NaN   
python               NaN           NaN  [experience]       NaN NaN   
flask                NaN           NaN  [experience]       NaN NaN   

                                 3  diff         man

In [290]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()
import glob
import textract
from pyresparser import ResumeParser
#print(porter.stem('javascript'))
str1='i have 2 years off experience in python and 4 years of experience in java'
str2='i have 3 years of experience in .net and jquery'
str3='i have high knowledge in python and basic knowledge in java'
str4='i have basic knowledge in c#.net but no knowledge in mvc'
technologies=['java','python','.net','jquery','c#.net','mvc','jquery','javascript','python','flask','sql',
              'asp.net','c#','machinelearning','artificialinteligence','ajax','vb.net','ado.net','neuralnetworks']
exp=['experience','knowledge','idea','language','develped','involved','good']
year=['year','month','years','months']
words=['machine learning','artificial inteligence','jave script','j query','neural networks']
shortcuts={'ai':'artificialinteligence','ml':'machinelearning','nlp':'naturallanguageprocessing'}

exp1=[]
year1=[]
technologies1=[]
for k in exp:
    exp1.append(porter.stem(k))
for k in year:
    year1.append(porter.stem(k))
for k in technologies:
    technologies1.append(porter.stem(k))   
#print('exp  :-',exp1,'\n','yeras :-',year1,'\n','technologies :-',technologies1,'\n\n\n')    
ll2=str4.split(' ')
files=[]
final={}
for file in glob.glob("/Users/pradeep/Documents/jupyter/profilesforproject/1.docx"):
    result={}
    data1 = ResumeParser(file).get_extracted_data()
    #print('##########################',data1)
    ll=[]
    text = textract.process(file)
    #print(text)
    name=(file.split('/')[-1]).split('.')[0]
    text = text.decode("utf-8")

    text=text.replace(',', ' ').replace('(',' ').replace(')',' ').replace(':',' ').replace('+',' ').replace('|',' ')
#print(text[15])
    my_text=text.lower().split('.\n')
#print(my_text[15])

    ll3=[]
    for i in my_text:
     kk=i.split()
     kk1=[]
     for value,term in enumerate(kk):
          #print(value,term)
          if value+2<len(kk):
              if kk[value]+' '+kk[value+1] in words:
                term=kk[value]+''+kk[value+1]
          if term in shortcuts.keys():
           kk1.append(porter.stem(shortcuts[term]))
          else:
           kk1.append(porter.stem(term))

     #print('kk1222233333333',kk1)           
     ll3.append(kk1)
        
    for ll2 in ll3:
     #print('ssssss',ll2)   
     for k,j in enumerate(ll2):
           #print('ooooo',j)
           for l in technologies1: 
            #print('oooooooooo',k,j,l)    
            if j==l:
                flag1='TRUE'
                flag2='TRUE'
                front=ll2[:k+1]
                rare=ll2[k+1:]
                #print('kkkkkkkkkkk',front,rare)
                for k1,k2 in enumerate(front[::-1]):
                    if k2 == 'year' and flag1=='TRUE':  
                      #print(j,front[-k1-2],front[-k1-1]) 
                      ll.append((j,round(float(front[-k1-2])*12,1)))  
                      flag1='FLASE'
                    if k2 == 'month' and flag1=='TRUE':  
                      #print(j,front[-k1-2],front[-k1-1]) 
                      ll.append((j,round(float(front[-k1-2])*1,1)))  
                      flag1='FLASE'                    
                    if k2 in exp1 and flag2=='TRUE':
                      #print(j,front[front.index(k)-1],front[front.index(k)]) 
                      #ll.append((j,'knowledge'))
                      #print(j,front[-k1-2])  
                      ll.append((j,8))
                      flag2='FLASE' 

                for r,k4 in enumerate(rare):
                    #print('k',k)
                    if k4=='year' and flag1=='TRUE':
                      #print(j,rare[r])
                      ll.append((j,round(float(rare[r-1])*12,2)))
                      flag1='FLASE'
                    if k4=='month' and flag1=='TRUE':
                      #print(j,rare[r])
                      ll.append((j,round(float(rare[r-1])*1,2)))
                      flag1='FLASE'                    
                    if k4 in exp1 and flag2=='TRUE':
                      #print(j,front[front.index(k)-1],front[front.index(k)]) 
                      #print(j,rare[r-1])
                      #ll.append((j,'knowledge'))
                      ll.append((j,8))                  
                      flag2='FLASE'                     
                                                     
    #print(ll) 
    for key,value in ll:
        if key not in result.keys():
            result[key]=value
        else:
            rr=result[key]
            if rr > value:
                pass
            else:
                result[key]=value
    #print(result) 
    #result['name']=data1['name']
    #result['email']=data1['email']
    #result['mobile_number']=data1['mobile_number']
    final[name]=result
#print(final) 
import pandas as pd
hello=pd.DataFrame.from_dict(final,orient='index')
print(hello)
hello['c#']=(hello['c#']-10).abs()
for i in hello.columns.values:
    print(i)
#hello=hello.iloc[(hello['c#']-2.0).abs()]
#print(pd.DataFrame.from_dict(hello))
#print(pd.DataFrame.from_dict(final))
print(hello['c#'].max())

   .net  c#  asp.net  mvc  jqueri  ado.net  javascript  sql  ajax  c#.net
1  48.0   8        8    8       8        8           8    8     8       8
.net
c#
asp.net
mvc
jqueri
ado.net
javascript
sql
ajax
c#.net
2


In [187]:

hello6={}
for i,v in final.items():
    hello2={}
    for j,k in v.items():
       hello2[j]=1
    hello6[i]=hello2 
df=pd.DataFrame.from_dict(hello6).replace(numpy.nan, 0 , regex=True).to_dict()  
#print(df)
import textract
from collections import Counter
technologies=['java','python','.net','jquery','c#.net','mvc','jquery','javascript','python','flask','sql',
              'asp.net','c#','azure','aws','agile','react','mysql','angular','kendo','machinelearning','artificialinteligence','ajax','vb.net','ado.net','neuralnetworks']
for file in glob.glob("/Users/pradeep/Documents/jupyter/profilesforproject/asp.net_job_desc1.docx"):
    text = textract.process(file)
    text = text.decode("utf-8")

    text=text.replace(',', ' ').replace('\n', ' ').replace('\xa0', ' ').replace('(',' ').replace(')',' ').replace(':',' ').replace('+',' ').replace('|',' ')
    my_text=text.lower().split(' ')
    hi=Counter(my_text)
#print(hi,type(hi))  
#print(my_text)
ll={}
list1=[]
for i,v in enumerate(my_text):
      if my_text[i]=="developer" and my_text[i-1] in technologies:
          list1.append(my_text[i-1])  
list1=list(set(list1))  
#print(list1)
for k,v in hi.items():
    if k in technologies:
        ll[k]=v
print('before',ll) 
final5={}
low=1
min=[2,3]
high=[4,5,6]
hello=[1,2,3,4,5,6]
for k,v in ll.items():
    if k in list1:
        final5[k]=5
    if v in high:
        final5[k]=4
    if v in min:
        final5[k]=2
    if v==1:
        final5[k]=1 
    if v not in hello:
        final5[k]=5
    if k in list1:
        final5[k]=5    
print('\n','final',final5,'\n')        
hello3={}
for i,v in df.items():
    ll=0
    for j,k in v.items() :
        if j in final5.keys():
         ll=ll+(k*final5[j])
    hello3[i]=ll    
print('jjjjjjjjj',hello3,'\n')   
print(sorted(hello3.items(), key=lambda x: x[1], reverse=True))

before {'asp.net': 3, 'mvc': 3, 'c#': 1, '.net': 1, 'ado.net': 1, 'mysql': 1, 'jquery': 1, 'angular': 2, 'react': 1, 'kendo': 1, 'azure': 1, 'aws': 1}

 final {'asp.net': 2, 'mvc': 5, 'c#': 1, '.net': 1, 'ado.net': 1, 'mysql': 1, 'jquery': 1, 'angular': 2, 'react': 1, 'kendo': 1, 'azure': 1, 'aws': 1} 

jjjjjjjjj {'4': 5.0, '1': 10.0, 'mani1': 0.0, '5': 8.0, 'job_description': 8.0, '2': 3.0, '3': 9.0, 'php_job_description': 1.0, 'diff': 0.0, 'asp': 10.0, 'mani2': 0.0} 

[('1', 10.0), ('asp', 10.0), ('3', 9.0), ('5', 8.0), ('job_description', 8.0), ('4', 5.0), ('2', 3.0), ('php_job_description', 1.0), ('mani1', 0.0), ('diff', 0.0), ('mani2', 0.0)]


In [ ]:
hello1={}
for i,v in final.items():
    hello2={}
    for j,k in v.items():
       hello2[j]=1
    hello1[i]=hello2    
import pandas as pd
#hello=pd.DataFrame.from_dict(hello1).to_dict()
df=pd.DataFrame.from_dict(hello1).replace(numpy.nan, 0 , regex=True).to_dict()
ex={'c#.net': 5, 'vb.net': 1, 'asp.net': 1.0, 'ado.net': 1.0, '.net': 1.0, 'c#': 1.0, 'sql': 1.0, 'javascript': 1.0, 'ajax': 1.0, 'mvc': 40.0, 'jqueri': 10.0, 'python': 100, 'flask': 60.0, 'machinelearn': 80.0, 'neuralnetwork': 2.0}
hello3={}
for i,v in df.items():
    ll=0
    for j,k in v.items():
        ll=ll+(k*ex[j])
    hello3[i]=ll    
print(hello3)        

In [219]:
import re
import spacy
from nltk.corpus import stopwords

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'B-TECH', 'B.TECH','BTECH','M-TECH', 'M.TECH', 'MTECH', 
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII','MCA','MBA'
        ]

def extract_education(resume_text):
    nlp_text = nlp(resume_text)
    print('hhhhhh',nlp_text.sents)
    for sent in nlp_text.sents:
        print('jjjjjjjjjj',sent.string.strip())
    # Sentence Tokenizer
    nlp_text = [sent.string.strip() for sent in nlp_text.sents]
    #print('llllllllll',nlp_text)
    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index]
    print(edu)
    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education
for file in glob.glob("/Users/pradeep/Documents/jupyter/profilesforproject/4.doc"):
    text = textract.process(file)
    text = text.decode("utf-8")
    print(extract_education(text))

hhhhhh <generator object at 0x12ea7d690>
jjjjjjjjjj Anil Kumar Kilari
jjjjjjjjjj Mobile: +91 9533392977
jjjjjjjjjj Email ID:
jjjjjjjjjj venkatanil.kilari@gmail.com
jjjjjjjjjj Career Objectives
jjjjjjjjjj Looking  for a challenging and Responsible position in  the  field  of
Microsoft Technologies and
jjjjjjjjjj have  the  flexibility  to  adapt  to  any  new
environment and work on any project wish to utilize this  experience  in  an
organization as part of them.
jjjjjjjjjj Professional Summary:
jjjjjjjjjj V Around 4 years of experience in analysis, designing, development and
      implementation of Windows and web based
jjjjjjjjjj applications, web services
jjjjjjjjjj using Microsoft technologies and its related technologies.
jjjjjjjjjj V Good experience on C#.Net, VB.net,
jjjjjjjjjj ASP.Net, ADO.Net,  .Net
jjjjjjjjjj Framework
      4.0/3.5.
jjjjjjjjjj V Good knowledge in Object Oriented Concepts.
jjjjjjjjjj V Good experience in Development of web based application  using  ASP.Net,
 

In [230]:
import nltk 
sentence = "B.SC (Computers)  Acharya Nagarjuna  University(ANU)       69.2%" 
token = nltk.word_tokenize(sentence) 
nltk.pos_tag(token)

[('B.SC', 'NNP'),
 ('(', '('),
 ('Computers', 'NNPS'),
 (')', ')'),
 ('Acharya', 'NNP'),
 ('Nagarjuna', 'NNP'),
 ('University', 'NNP'),
 ('(', '('),
 ('ANU', 'NNP'),
 (')', ')'),
 ('69.2', 'CD'),
 ('%', 'NN')]